In [11]:
import torch
from project_helper import TweetData
import numpy as np
import pandas as pd
import os

In [12]:
tweet_data = TweetData()

In [14]:
daily_tweets = tweet_data.clean_tweets #[pd.to_datetime(tweet_data.clean_tweets.after4_date)
                                       #<= pd.to_datetime(daily_df.index[-1])]
daily_tweets.after4_date = pd.to_datetime(daily_tweets.after4_date)

In [16]:
daily_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


In [2]:
model = torch.load('models/trump_rnn_1911.net')

In [50]:
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [4]:
embeddings.shape

(4574, 100)

In [5]:
arches = os.listdir('models')
arches[0:5]

['trump_rnn_1702.net',
 'trump_rnn_1703.net',
 'trump_rnn_1908.net',
 'trump_rnn_1707.net',
 'trump_rnn_1712.net']

In [6]:
embedding_list = []
for arch in arches:
    model = torch.load('models/{}'.format(arch))
    embedding_list.append(model.in_embed.weight.to('cpu').data.numpy())
    

In [7]:
embedding_list[0].shape

(2576, 100)

In [8]:
data = TweetData('data/opt/trump_archive_db_1911.csv')
words = data.words
vocab_to_int, int_to_vocab = data.vocab_to_int, data.int_to_vocab
int_words = data.int_words

In [33]:
embedding_list[-1]

array([[-0.07983071,  0.13481925, -0.3339518 , ...,  0.07891949,
        -0.03683575, -0.06613905],
       [-0.4210904 ,  0.02706584, -0.1898706 , ..., -0.17446454,
        -0.08358873, -0.03138035],
       [-0.22094908, -0.20108747, -0.04167196, ...,  0.11216594,
         0.00401473,  0.15296404],
       ...,
       [-0.38425526, -0.3069386 ,  0.7174769 , ..., -0.4966805 ,
        -0.22396143, -0.90884376],
       [-1.4828042 ,  0.38311642,  0.23013367, ...,  0.69838697,
         0.10621499,  0.49422497],
       [-0.72725123,  0.19940047, -0.78007627, ...,  0.00616149,
         0.37083137, -0.43986282]], dtype=float32)

In [23]:
daily_tweets.after4_date

timestamp
2019-11-17 19:57:12-06:00     True
2019-11-17 19:56:02-06:00     True
2019-11-17 19:49:47-06:00     True
2019-11-17 19:47:32-06:00     True
2019-11-17 19:30:09-06:00     True
                             ...  
2009-05-12 14:07:28-05:00    False
2009-05-08 20:40:15-05:00    False
2009-05-08 13:38:08-05:00    False
2009-05-05 01:00:10-05:00    False
2009-05-04 18:54:02-05:00    False
Name: after4_date, Length: 28813, dtype: bool

In [202]:
tweet_embeddings = {}
dtws = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
tweet_embeddings_np = np.zeros([dtws.shape[0],100])

for i, tweet in enumerate(dtws):
    embed_sum = np.zeros(100) 
    tw = tweet.split()
    for word in tw:
        try:
            if vocab_to_int[word] >= 100:
                embed_sum += embedding_list[-1][vocab_to_int[word]]

        except:
            pass
    tweet_embeddings[i] = embed_sum
    tweet_embeddings_np[i,:] = embed_sum
            
        

In [204]:
tweet_embeddings_np

array([[-1.47077961, -1.99534133, -9.21356507, ...,  0.14838102,
         5.38709498, -0.47451643],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-3.74675519,  0.21890989, -3.47137829, ...,  5.13478435,
         2.4891945 ,  2.45625968],
       ...,
       [ 0.05385559, -0.20527905, -0.19967043, ...,  0.48037067,
         0.91894266, -0.54557519],
       [-1.58845638,  0.05823702, -0.46339556, ...,  1.2611837 ,
         0.55581431, -1.1208628 ],
       [-0.48648791, -0.79444802,  0.18657702, ...,  0.52738795,
         0.239182  , -0.11716399]])

In [186]:
embed_vectors.shape

torch.Size([4000, 100])

In [239]:
tw = 1000

embedding = model.in_embed
embed_vectors = embedding.weight
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes)).topk(10)
for wd in list(wds[1][0].numpy()):
    if wd> 100 :
        print(int_to_vocab[wd])
tweet_after[tw]

collusion
incredible
friday


'democrats wrote to the ukrainian government in may urging it to continue investigations into president donald trumps alleged collusion with russia in the presidential campaign collusion later found not to exist '

In [263]:
wds

tensor([[23.3585,  0.0000, 21.8033,  ..., 12.9470, 15.9481, 14.7619]])

In [266]:
tw = 1467

embedding = model.in_embed
embed_vectors = torch.FloatTensor(tweet_embeddings_np)
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes))
wds[wds != wds] = 0
print(wds.topk(10))
for i in wds.topk(10).indices.numpy():
    for tweet_range in i:
        print(tweet_after[tweet_range])
        print('')


torch.return_types.topk(
values=tensor([[32.8715, 28.3099, 28.2627, 27.8419, 27.8317, 27.8011, 27.7800, 27.6092,
         27.5333, 27.5282]]),
indices=tensor([[1467, 1673, 1574, 1575, 1200, 5555, 4169, 2394, 6768, 1474]]))
the federal reserve loves watching our manufacturers struggle with their exports to the benefit of other parts of the world has anyone looked at what almost all other countries are doing to take advantage of the good old usa our fed has been calling it wrong for too long 

spread is way too much as other countries say thank you to clueless jay powell and the federal reserve germany and many others are playing the game crazy inverted yield curve we should easily be reaping big rewards amp gains but the fed is holding us back we will win 

doing great with china and other trade deals the only problem we have is jay powell and the fed hes like a golfer who cant putt has no touch big u s growth if he does the right thing big cut but dont count on him so far he has called

tonight we forcefully condemn the blatant corruption of the democrat party the fake news media and the rogue bureaucrats of the deep state the only message these radicals will understand is a crushing defeat on november 

became number in world amp independent in energy will soon have record number of judges sc justices done more than any president in first years despite phony amp fraudulent witch hunt illegally led against him win on mueller report mueller testimony amp james comey 

congratulations to maxine waters whose crazy rants have made her together with nancy pelosi the unhinged face of the democrat party together they will make america weak again but have no fear america is now stronger than ever before and im not going anywhere 

the fake news media keeps saying we havent built any new wall below is a section just completed on the border anti climbing feature included very high strong and beautiful also many miles already renovated and in service 

why didnt robert mueller a

In [235]:

tweet_after[800]

'my daughter ivanka will be on tonight on at p m following the great at enjoy '

In [236]:
tweet_after[9335]

'my daughter ivanka has been treated so unfairly by she is a great person always pushing me to do the right thing terrible '

In [237]:
tweet_after[4955]

'will be interviewed tonight by on at p m eastern enjoy '

In [238]:
tweet_after[2434]

'i will be interviewed live tonight by on p m enjoy '